# The [Modified Benedict-Webb-Rubin equation of state](https://en.wikipedia.org/wiki/Benedict–Webb–Rubin_equation) 
[Johnson, Zllweg, Gubbins, 
"The Lennard-Jones equation of state revisited",
Molecular Physics, 1993](Johnson_Lennard-Jones_1993.pdf)

implementing the parameters from Table 10

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

/home/schowell/data/myPrograms/anaconda3/envs/sassie/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Loading BokehJS ...

**Reduced Units:**
  - residual Helmholtz free energy: $A^*_r = A_r/N\epsilon$
  - pressure: $P^* = P\sigma^3/\epsilon$
  - temperature: 
  
| Quantity    | Reduced Units      | ------- Real Units --------      |
|-------------|--------------------|----------------------------------|
| temperature | $T^*=1$            | $T=119.8$ K                      |
| density     | $\rho^*=1.0$       | $\rho=1680$ kg/m$^3$             |
| time        | $\Delta t^*=0.005$ | $\Delta t=1.09\times 10^{-14}$ s |
| pressure    | $P^*=1$            | $P=41.9$ MPa                     |

$T^*=2.0$, to match Frenkel & Smit, figure 6.2

In [3]:
T = 2.0

Modified Benedict-Webb-Rubin (MBWR) equation of state, Johnson et al. 1993, eq. (7):
$$ P^* = \rho^* T^* + \sum_{i=1}^8 a_i \rho^{*(i+1)} + \exp\left(-\gamma\rho^{*2}\right)\sum_{i=1}^{6}b_i \rho^{*(2i+1)} $$

1 nonlinear parameter:
  - $\gamma = 3$

In [4]:
gamma = 3

In [5]:
parameters = np.loadtxt('mbwr_eos.dat')
x = np.empty(len(parameters)+1) 
x[1:] = parameters[:, 1] # use 1 indexing for clarity
# print x

First 19 parameters, $a_1, \dots, a_8$:
  - $a_1 = x_1 T^* + x_2 \sqrt{T} + x_3 + x_4/T^* + x_5/T^{*2}$
  - $a_2 = x_6 T^* + x_7 + x_8/T^* + x_9/T^{*2}$
  - $a_3 = x_{10} T^* + x_{11} + x_{12}/T^*$
  - $a_4 = x_{13}$
  - $a_5 = x_{14}/T^* + x_{15}/T^{*2}$
  - $a_6 = x_{16}/T^*$
  - $a_7 = x_{17}/T^* + x_{18}/T^{*2}$
  - $a_8 = x_{19}/T^{*2}$

In [6]:
a1 = x[1]*T + x[2]*np.sqrt(T) + x[3] + x[4]/T + x[5]/T**2
a2 = x[6]*T + x[7] + x[8]/T + x[9]/T**2
a3 = x[10]*T + x[11] + x[12]/T
a4 = x[13]
a5 = x[14]/T + x[15]/T**2
a6 = x[16]/T
a7 = x[17]/T + x[18]/T**2
a8 = x[19]/T**2
a = np.empty(8)
a[:] = [a1, a2, a3, a4, a5, a6, a7, a8]

Last 13 parameters, $b_1, \dots, b_6$:
  - $b_1 = x_{20}/T^{*2} + x_{21}/T^{*3}$
  - $b_2 = x_{22}/T^{*2} + x_{23}/T^{*4}$
  - $b_3 = x_{24}/T^{*2} + x_{25}/T^{*3}$
  - $b_4 = x_{26}/T^{*2} + x_{27}/T^{*4}$
  - $b_5 = x_{28}/T^{*2} + x_{29}/T^{*3}$
  - $b_6 = x_{30}/T^{*2} + x_{31}/T^{*3} + x_{32}/T^{*4}$

In [7]:
b1 = x[20]/T**2 + x[21]/T**3
b2 = x[22]/T**2 + x[23]/T**4
b3 = x[24]/T**2 + x[25]/T**3
b4 = x[26]/T**2 + x[27]/T**4
b5 = x[28]/T**2 + x[29]/T**3
b6 = x[30]/T**2 + x[31]/T**3 + x[32]/T**4
b = np.empty(6)
b[:] = [b1, b2, b3, b4, b5, b6]

Modified Benedict-Webb-Rubin (MBWR) equation of state, Johnson et al. 1993, eq. (7):
$$ P^* = \rho^* T^* + \sum_{i=1}^8 a_i \rho^{*(i+1)} + \exp\left(-\gamma\rho^{*2}\right)\sum_{i=1}^{6}b_i \rho^{*(2i+1)} $$

In [8]:
density = np.linspace(0.1, 0.9)
f = np.exp(-gamma * density**2)

In [9]:
pressure200 = (
    density * T + 
    a1 * density**2 + 
    a2 * density**3 +
    a3 * density**4 + 
    a4 * density**5 +
    a5 * density**6 +
    a6 * density**7 +
    a7 * density**8 +
    a8 * density**9 +
    f * b1 * density**3 +
    f * b2 * density**5 +
    f * b3 * density**7 +
    f * b4 * density**9 +
    f * b5 * density**11 +
    f * b6 * density**13    
)

Validating implentation against Frenkel Smit, Figure 6.2

In [11]:
eos = np.loadtxt('fs_eos_t2.csv', delimiter=',')
mc = np.loadtxt('fs_mc_t2.csv', delimiter=',')
md = np.loadtxt('fs_md_t2.csv', delimiter=',')

In [12]:
p = figure(width=500, height=500, x_axis_label='Density',
           y_axis_label='Pressure')
p.line(density, pressure200, legend='eos Johnson', line_width=2)
p.line(eos[:,0], eos[:,1], legend='eos F&S', line_dash=[2, 2], 
       line_width=2, color='firebrick')
p.circle(mc[:,0], mc[:,1], legend='MC F&S', size=12, 
         line_width=2, fill_alpha=0, line_color='orange')
p.square(md[:,0], md[:,1], legend='MD F&S', size=12, 
         line_width=2, fill_alpha=0, line_color='yellow')
p.legend.location = 'top_left'
show(p)

Repeat using $T^*=0.71$ ($T=85$ K), to match Liquid Argon data published by [Yarnell et al.](Yarnell_Structure_1973.pdf)

In [13]:
T = 0.71

In [14]:
a1 = x[1]*T + x[2]*np.sqrt(T) + x[3] + x[4]/T + x[5]/T**2
a2 = x[6]*T + x[7] + x[8]/T + x[9]/T**2
a3 = x[10]*T + x[11] + x[12]/T
a4 = x[13]
a5 = x[14]/T + x[15]/T**2
a6 = x[16]/T
a7 = x[17]/T + x[18]/T**2
a8 = x[19]/T**2

In [15]:
b1 = x[20]/T**2 + x[21]/T**3
b2 = x[22]/T**2 + x[23]/T**4
b3 = x[24]/T**2 + x[25]/T**3
b4 = x[26]/T**2 + x[27]/T**4
b5 = x[28]/T**2 + x[29]/T**3
b6 = x[30]/T**2 + x[31]/T**3 + x[32]/T**4

In [16]:
pressure071 = (
    density * T + 
    a1 * density**2 + 
    a2 * density**3 +
    a3 * density**4 + 
    a4 * density**5 +
    a5 * density**6 +
    a6 * density**7 +
    a7 * density**8 +
    a8 * density**9 +
    f * b1 * density**3 +
    f * b2 * density**5 +
    f * b3 * density**7 +
    f * b4 * density**9 +
    f * b5 * density**11 +
    f * b6 * density**13    
)

In [17]:
p = figure(width=500, height=500, x_axis_label='Density', 
           y_axis_label='Pressure')
p.line(density, pressure200, legend="T=2.00")
p.line(density, pressure071, legend="T=0.71", line_color="firebrick")
p.legend.location='top_left'
show(p)

While the $T=2.0$ case matches Figure 6.2 in Frenkel & Smit, I am skeptical about the $T=0.71$ case.  I do not understand a negative pressure.  Their article says, "Parameters for the MBWR equation of state for the LJ fluid have been regressed for the temperature range $0.7\leq T^*\leq 6$ and covering the entire fluid range of densities."

### Load in Experimental Data

In [34]:
def rm_par(astr):
    return float(astr.split(b'(')[0])

In [36]:
import sys; print(sys.executable)

/home/schowell/data/myPrograms/anaconda3/envs/sassie/bin/python


In [37]:
data=np.genfromtxt('state_point.dat', dtype=None, names=True, converters={2:rm_par, 3:rm_par})

In [52]:
t20 = data[data['T']==2.0]
t07 = data[data['T']==0.7]

In [53]:
p = figure(width=500, height=500, x_axis_label='Density', 
           y_axis_label='Pressure')
p.line(density, pressure200, legend="T=2.00 EOS")
p.line(density, pressure071, legend="T=0.71 EOS", line_color="firebrick")
p.circle(t20['p'], t20['P'], legend="T=2.00 Data")
p.circle(t07['p'], t07['P'], legend="T=0.70", color="firebrick")
p.legend.location='top_left'
show(p)

So the curve matches the experimental data, and the experimental curve even goes negative.